# Preliminaries

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.9 MB/s eta 0:00:00


In [3]:
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-63e_3yq7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-63e_3yq7
  Resolved https://github.com/huggingface/accelerate to commit 7b4d12623a1847bf7f749d6824017683922339eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.21.0.dev0-py3-none-any.whl size=231043 sha256=e274037a89a64f2bb98ce4c58908edcc0130dc257acab20dd69d925133f25e38
  Stored in directory: /tmp/pip-ephem-wheel-cache-dhmqosmn/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [4]:
from os import listdir

import pandas as pd
import numpy as np
import random
import re

import torch
from torch.utils.data import Dataset, random_split
from transformers import TrainingArguments, Trainer, GPTNeoForCausalLM, GPT2TokenizerFast
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Loading dataset and preprocessing

In [5]:
LYRICS_GENRE = 'Rock'

In [6]:
data = pd.read_csv("/content/drive/MyDrive/NLP Finetuning/Genre Datasets/en_"+LYRICS_GENRE+"_Lyrics.csv")

print(data.shape)
data.head()

(63553, 3)


,Unnamed: 0,Lyric,Genres
0,31764,"Wise men say, only fools rush in\nBut I can't ...",Rock
1,31767,We're caught in a trap\nI can't walk out\nBeca...,Rock
2,31770,"Maybe I didn't treat you,\nQuite as good as I ...",Rock
3,31773,"Love me tender, love me sweet,\nNever let me g...",Rock
4,31776,And now the end is near\nAnd so I face the fin...,Rock


In [7]:
data['length'] = data['Lyric'].apply(lambda x:len(str(x).split()))
data = data[data['length'] <= 200]

data = data.drop(['Unnamed: 0', 'Genres', 'length'], axis=1)

In [9]:
data = data.head(2000)

In [10]:
torch.cuda.empty_cache()

tokenizer = GPT2TokenizerFast.from_pretrained('EleutherAI/gpt-neo-125m', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m')
model.resize_token_embeddings(len(tokenizer))

torch.cuda.empty_cache()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
if torch.cuda.is_available():
  device = torch.device('cuda')

else:
  device = torch.device('cpu')

print(device)

cuda


In [12]:
!nvidia-smi

Wed Jun 21 08:18:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
train, test = train_test_split(data['Lyric'], test_size=0.15, shuffle=True, random_state=SEED)

# Shaping the dataset

In [14]:
class ShapingDataset(Dataset):

    def __init__(self, texts):
        self.texts = texts
        self.tokenizer = tokenizer
        self.labels = []
        self.input_ids = []
        self.attention_masks = []
        self.max_length = max([len(re.findall('\w+', x)) for x in data['Lyric']]) # Lyric with most words.
        for text in texts:
            dici = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
            self.input_ids.append(dici['input_ids'][0])
            self.attention_masks.append(dici['attention_mask'][0])

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx]

In [15]:
train = ShapingDataset(train)
test = ShapingDataset(test)

# Training model

In [16]:
training_args = TrainingArguments(
    output_dir = '/content/outputs/',
    logging_dir = '/content/logs/',
    num_train_epochs = 20,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    logging_steps = 10000,
    save_steps = 10000,
    learning_rate = float('2e-4'),
    warmup_steps = 20)

In [17]:
model.to(device)

trainer = Trainer(model=model, args=training_args,  train_dataset=train, eval_dataset=test, data_collator = lambda data: {
    'input_ids': torch.stack([f[0] for f in data]),
    'attention_mask': torch.stack([f[1] for f in data]),
    'labels': torch.stack([f[0] for f in data])})

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10000,1.123600
20000,0.115800
30000,0.030500


TrainOutput(global_step=34000, training_loss=0.37564003372192384, metrics={'train_runtime': 3988.6396, 'train_samples_per_second': 8.524, 'train_steps_per_second': 8.524, 'total_flos': 4371137519616000.0, 'train_loss': 0.37564003372192384, 'epoch': 20.0})

# Saving the trained model

In [19]:
saved_root = "/content/drive/MyDrive/NLP Finetuning/Finetuned Models/"+LYRICS_GENRE+" Lyrics Generator"

In [20]:
trainer.save_model(saved_root)

In [21]:
model = GPTNeoForCausalLM.from_pretrained(saved_root)

In [ ]:
model.to('cuda')
generated = tokenizer(' ', return_tensors = 'pt').input_ids.cuda()

In [ ]:
sample_outputs = model.generate(generated, do_sample=True, top_k=10, max_length=100, top_p=0.95, temperature=2., num_return_sequences=5)

for i, sample_output in enumerate(sample_outputs):
  text = tokenizer.decode(sample_output.tolist())
  regex_padding = re.sub('<|pad|>', '', text)
  regex_barra = re.sub('[|+]', '', regex_padding)
  space = re.sub('[ +]', ' ', regex_barra)
  result = re.sub('[\n](2, )', '\n', space)
  print("\n==== Generated lyric {}:\n\n{}".format(i+1, result))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



==== Generated lyric 1:

  I was blind,  
Look behind me,  
You knew how I long to hold you tight
Just once again
I was blind,  
Without a prayer,
Without a love of my own

There will be peace in theearcher's mind, some day 
I guess it will be,
One tomorrows may be over;
One to lead me,  
On your part, I will have found a heart that is true

==== Generated lyric 2:

  river, it can't be found on a map, it's where the sea was never found,
To the cross of Abraham

==== Generated lyric 3:

  children airstrike
.) )]  lbs
�

==== Generated lyric 4:

  baby, if it's too early, send the baby
            Tell me.  In the evening,
             When's Sunday in the land of honey?

Blue moon, come and take my baby
Up to the open arms of the learned
                     "Hey, I got

==== Generated lyric 5:

  river it would lead me to Bumble Bee Cafe
Or the bumble hoh boy could lead me to
Bumble Bee Cafe

Bumble Bee Cafe '..eworks.).))].
